Run [circular-tank-detector](https://github.com/PlatformStories/circular-tank-detector) on a pan-sharpened image of Fujairah, United Arab Emirates, with models 1 and 2. Details on how models 1 and 2 were obtained can be found in [this blog post](http://gbdxstories.digitalglobe.com/circular-tanks/).

In [1]:
# Set credentials

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

In [13]:
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/circular-tank-detector/'

from os.path import join
import uuid

output_location = join('platform-stories/trial-runs', str(uuid.uuid4()))

In [4]:
# Deploy models 1 and 2 with a minimum size of 50m2

ctd1 = gbdx.Task('circular-tank-detector')
ctd1.inputs.ps_image = join(input_location, 'ps-image')
ctd1.inputs.min_size = '50' 
ctd1.inputs.model = join(input_location, 'models', 'unsupervised')

# model 2 is the default model and is built into the task; no need to specify it as input
ctd2 = gbdx.Task('circular-tank-detector')
ctd2.inputs.ps_image = join(input_location, 'ps-image')
ctd2.inputs.min_size = '50' 

wf = gbdx.Workflow([ctd1, ctd2])
wf.savedata(ctd1.outputs.detections, join(output_location, '1'))
wf.savedata(ctd2.outputs.detections, join(output_location, '2'))

wf.execute()

u'4713005137257746784'

In [10]:
wf.status

{u'event': u'succeeded', u'state': u'complete'}